In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
data_path = r"D:\agribot\datas/"
dir_loader = DirectoryLoader(
    data_path,
    glob='*.pdf',
    loader_cls=PyPDFLoader
)
docs = dir_loader.load()
print("PDF'S Loaded")

PDF'S Loaded


In [3]:
txt_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 500
)
inp_txt = txt_splitter.split_documents(docs)
print("Data Chunks Created")

Data Chunks Created


In [5]:
hfembeddings = HuggingFaceEmbeddings(
    model_name = "thenlper/gte-large",
    model_kwargs = {'device':'cuda'}
)
print("Embedding Created")

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Embedding Created


In [6]:
db = FAISS.from_documents(inp_txt,hfembeddings)
db.save_local(r"D:\agribot\faiss\agri_data")
print("Vector Store Creation Completed and Stored Locally")

d:\newenv\lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Vector Store Creation Completed and Stored Locally


# **inference**

In [1]:
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers , HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [2]:
PROMPT_TEMPLATE = '''
With the information provided try to answer the question. 
If you cant answer the question based on the information either say you cant find an answer or unable to find an answer.
So try to understand in depth about the context and answer only based on the information provided. Dont generate irrelevant answers

Context: {context}
Question: {question}
Do provide only helpful answers

Helpful answer:
'''
INP_VARS = ['context', 'question']
custom_prompt_template = PromptTemplate(
    template = PROMPT_TEMPLATE,
    input_variables = INP_VARS
)

In [3]:
llm = CTransformers(
    model = r"D:\agribot\llama-2-7b-chat.ggmlv3.q4_1.bin",
    model_type="llama",
    max_new_tokens = 512,
    temperature = 0.2,
    callbacks=[StreamingStdOutCallbackHandler()],
    gpu_layers=0
)

In [5]:
hfembeddings = HuggingFaceEmbeddings(
    model_name = "thenlper/gte-large",
    model_kwargs = {'device':'cuda'}
)
print("Embedding Created")

Embedding Created


In [6]:
vector_db = FAISS.load_local(r"D:\agribot\faiss\agri_data/", hfembeddings,allow_dangerous_deserialization=True)

In [7]:
retrieval_qa_chain = RetrievalQA.from_chain_type(
                                llm=llm,
                                chain_type="stuff",
                                retriever=vector_db.as_retriever(search_kwargs={'k': 1}),
                                return_source_documents=True,
                                chain_type_kwargs={"prompt": custom_prompt_template}
                            )

In [8]:
user_input = "Which of the parts of the plant is of greatest importance to the farmer or any plant grower?"
prompt = {'query': user_input}
model_out = retrieval_qa_chain(prompt)
answer = model_out['result']
print(answer)

d:\newenv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
d:\newenv\lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


The root system is the most important part of a plant for a farmer or plant grower. The root system absorbs water and nutrients from the soil, which are then transported throughout the plant to support growth and development. Without a healthy root system, a plant cannot survive, let alone thrive. While other parts of the plant, such as leaves, stems, and flowers, are important for photosynthesis and reproduction, they are dependent on the root system for water and nutrients. Therefore, it is crucial that farmers and plant growers prioritize the health and well-being of the root system in order to produce a bountiful harvest.The root system is the most important part of a plant for a farmer or plant grower. The root system absorbs water and nutrients from the soil, which are then transported throughout the plant to support growth and development. Without a healthy root system, a plant cannot survive, let alone thrive. While other parts of the plant, such as leaves, stems, and flowers, 